In [ ]:
```json
{
 "cells": [
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "# Stellar Forge Simulator: Data Exploration"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 1. Setup and Imports"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "import pandas as pd\n",
    "import numpy as np\n",
    "import matplotlib.pyplot as plt\n",
    "import seaborn as sns\n",
    "import json\n",
    "from sklearn.model_selection import train_test_split\n",
    "from sklearn.linear_model import LinearRegression\n",
    "from sklearn.ensemble import RandomForestRegressor\n",
    "from sklearn.mixture import GaussianMixture\n",
    "from sklearn.preprocessing import StandardScaler\n",
    "from sklearn.metrics import mean_squared_error, r2_score\n",
    "\n",
    "# Set plot style\n",
    "sns.set_theme(style=\"whitegrid\")\n",
    "plt.rcParams['figure.figsize'] = (10, 6)"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 2. Data Generation (Synthetic)"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "Since we don't have real simulation logs yet, we'll generate synthetic data representing planets in various star systems. This data will mimic properties that might be generated procedurally or used by the ML model."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "def generate_synthetic_planet_data(num_planets=1000, seed=42):\n",
    "    np.random.seed(seed)\n",
    "    \n",
    "    # Simulate properties based on orbital distance (AU)\n",
    "    orbital_distance = np.random.uniform(0.1, 50, num_planets) # Astronomical Units\n",
    "    \n",
    "    # Mass (Earth masses) - tends to increase with distance initially, then varies\n",
    "    log_distance_factor = np.log1p(orbital_distance)\n",
    "    mass = np.random.lognormal(mean=np.log(0.1 + log_distance_factor * 0.5), sigma=1.0, size=num_planets) + np.random.uniform(0.01, 5, num_planets)\n",
    "    mass = np.clip(mass, 0.01, 1000) # Clip unrealistic values\n",
    "    \n",
    "    # Radius (Earth radii) - correlated with mass (roughly M^(1/3))\n",
    "    radius_base = (mass**(1/3)) * np.random.normal(1.0, 0.2, num_planets)\n",
    "    radius = np.clip(radius_base, 0.1, 20) # Clip unrealistic values\n",
    "    \n",
    "    # Atmospheric Density (relative scale 0-1) - higher for larger planets, decreases further out\n",
    "    atmosphere_base = (mass / (radius**2)) * np.exp(-orbital_distance / 25) * np.random.normal(1.0, 0.3, num_planets)\n",
    "    atmospheric_density = np.clip(atmosphere_base / np.max(atmosphere_base) * np.random.uniform(0.5, 1.0), 0, 1) # Normalize and clip\n",
    "    \n",
    "    # Color (simplified categorical or hex)\n",
    "    colors = ['#A0522D', '#D2B48C', '#B0C4DE', '#FFEBCD', '#87CEEB', '#4682B4', '#F4A460', '#E0FFFF']\n",
    "    planet_color = np.random.choice(colors, num_planets)\n",
    "    \n",
    "    # Star Mass (Solar masses) - Assume multiple systems\n",
    "    star_mass = np.random.choice([0.8, 1.0, 1.2, 1.5, 2.0], num_planets, p=[0.2, 0.4, 0.2, 0.1, 0.1]) * np.random.normal(1.0, 0.05, num_planets)\n",
    "    \n",
    "    df = pd.DataFrame({\n",
    "        'planet_id': range(num_planets),\n",
    "        'star_mass_solar': star_mass,\n",
    "        'orbital_distance_au': orbital_distance,\n",
    "        'mass_earth': mass,\n",
    "        'radius_earth': radius,\n",
    "        'atmospheric_density': atmospheric_density,\n",
    "        'color_hex': planet_color\n",
    "    })\n",
    "    \n",
    "    return df\n",
    "\n",
    "planets_df = generate_synthetic_planet_data(num_planets=1500, seed=42)"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 3. Data Loading and Inspection"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "print(\"First 5 rows of the dataset:\")\n",
    "planets_df.head()"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "print(\"\\nDataset Information:\")\n",
    "planets_df.info()"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "print(\"\\nSummary Statistics:\")\n",
    "planets_df.describe()"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "print(\"\\nMissing Values:\")\n",
    "planets_df.isnull().sum()"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 4. Exploratory Data Analysis (EDA)"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "### 4.1 Univariate Analysis (Distributions)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "numerical_cols = ['star_mass_solar', 'orbital_distance_au', 'mass_earth', 'radius_earth', 'atmospheric_density']\n",
    "\n",
    "for col in numerical_cols:\n",
    "    plt.figure(figsize=(10, 5))\n",
    "    sns.histplot(planets_df[col], kde=True, bins=30)\n",
    "    plt.title(f'Distribution of {col}')\n",
    "    plt.xlabel(col)\n",
    "    plt.ylabel('Frequency')\n",
    "    plt.show()"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Log transform for skewed features (mass, radius)\n",
    "plt.figure(figsize=(12, 5))\n",
    "plt.subplot(1, 2, 1)\n",
    "sns.histplot(np.log1p(planets_df['mass_earth']), kde=True, bins=30)\n",
    "plt.title('Log Distribution of mass_earth')\n",
    "plt.xlabel('Log(1 + Mass (Earth))')\n",
    "\n",
    "plt.subplot(1, 2, 2)\n",
    "sns.histplot(np.log1p(planets_df['radius_earth']), kde=True, bins=30)\n",
    "plt.title('Log Distribution of radius_earth')\n",
    "plt.xlabel('Log(1 + Radius (Earth))')\n",
    "\n",
    "plt.tight_layout()\n",
    "plt.show()"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "plt.figure(figsize=(10, 6))\n",
    "sns.countplot(y=planets_df['color_hex'], order = planets_df['color_hex'].value_counts().index)\n",
    "plt.title('Distribution of Planet Colors (Hex)')\n",
    "plt.xlabel('Count')\n",
    "plt.ylabel('Color Hex Code')\n",
    "plt.show()"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "### 4.2 Bivariate Analysis (Relationships)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "sns.pairplot(planets_df[numerical_cols])\n",
    "plt.suptitle('Pair Plot of Numerical Features', y=1.02)\n",
    "plt.show()"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "plt.figure(figsize=(8, 6))\n",
    "sns.scatterplot(data=planets_df, x='mass_earth', y='radius_earth', alpha=0.6)\n",
    "plt.title('Planet Mass vs. Radius')\n",
    "plt.xlabel('Mass (Earth Masses)')\n",
    "plt.ylabel('Radius (Earth Radii)')\n",
    "# Log scale can be useful here too\n",
    "# plt.xscale('log')\n",
    "# plt.yscale('log')\n",
    "plt.show()"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "plt.figure(figsize=(10, 6))\n",
    "sns.scatterplot(data=planets_df, x='orbital_distance_au', y='atmospheric_density', hue='mass_earth', size='radius_earth', sizes=(20, 200), alpha=0.7, palette='viridis')\n",
    "plt.title('Atmospheric Density vs. Orbital Distance (Colored by Mass, Sized by Radius)')\n",
    "plt.xlabel('Orbital Distance (AU)')\n",
    "plt.ylabel('Atmospheric Density (Relative)')\n",
    "plt.show()"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "plt.figure(figsize=(10, 6))\n",
    "sns.scatterplot(data=planets_df, x='orbital_distance_au', y='mass_earth', hue='star_mass_solar', alpha=0.7, palette='coolwarm')\n",
    "plt.title('Planet Mass vs. Orbital Distance (Colored by Star Mass)')\n",
    "plt.xlabel('Orbital Distance (AU)')\n",
    "plt.ylabel('Mass (Earth Masses)')\n",
    "plt.yscale('log') # Log scale often better for mass\n",
    "plt.show()"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 5. Statistical Analysis"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "correlation_matrix = planets_df[numerical_cols].corr()\n",
    "\n",
    "plt.figure(figsize=(10, 8))\n",
    "sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt=\".2f\")\n",
    "plt.title('Correlation Matrix of Numerical Features')\n",
    "plt.show()\n",
    "\n",
    "print(\"\\nCorrelation Matrix:\")\n",
    "print(correlation_matrix)"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "Key observations from correlation:\n",
    "- Mass and Radius show a positive correlation, as expected.\n",
    "- Atmospheric density shows some positive correlation with mass and negative with orbital distance, matching our generation logic.\n",
    "- Star mass doesn't strongly correlate with individual planet properties in this simple generation, but might influence system-wide patterns."
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 6. Feature Engineering Experiments"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Calculate Planet Density (relative to Earth units)\n",
    "# Density = Mass / Volume; Volume = (4/3) * pi * R^3\n",
    "# Need to be careful about units if using real constants, here relative units are fine.\n",
    "planets_df['volume_relative'] = (4/3) * np.pi * (planets_df['radius_earth']**3)\n",
    "# Avoid division by zero or tiny volumes if radius is very small\n",
    "planets_df['density_relative'] = np.where(planets_df['volume_relative'] > 1e-6,\n",
    "                                        planets_df['mass_earth'] / planets_df['volume_relative'],\n",
    "                                        np.nan) # Assign NaN or a large number for tiny volumes\n",
    "\n",
    "# Calculate Surface Gravity (relative to Earth)\n",
    "# g ~ M / R^2\n",
    "planets_df['surface_gravity_relative'] = np.where(planets_df['radius_earth'] > 1e-6,\n",
    "                                                planets_df['mass_earth'] / (planets_df['radius_earth']**2),\n",
    "                                                np.nan)\n",
    "\n",
    "# Approximate Orbital Period (using simplified Kepler's 3rd Law: P^2 ~ a^3 / M_star)\n",
    "# P^2 proportional to a^3 / M_star => P proportional to sqrt(a^3 / M_star)\n",
    "planets_df['orbital_period_relative'] = np.sqrt((planets_df['orbital_distance_au']**3) / planets_df['star_mass_solar'])\n",
    "\n",
    "print(\"DataFrame with engineered features:\")\n",
    "planets_df[['planet_id', 'density_relative', 'surface_gravity_relative', 'orbital_period_relative']].head()"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Explore new features\n",
    "engineered_features = ['density_relative', 'surface_gravity_relative', 'orbital_period_relative']\n",
    "planets_df[engineered_features].describe()"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "plt.figure(figsize=(18, 5))\n",
    "for i, col in enumerate(engineered_features):\n",
    "    plt.subplot(1, 3, i + 1)\n",
    "    # Handle potential NaN/Inf values before plotting\n",
    "    feature_data = planets_df[col].replace([np.inf, -np.inf], np.nan).dropna()\n",
    "    sns.histplot(feature_data, kde=True, bins=30)\n",
    "    plt.title(f'Distribution of {col}')\n",
    "    plt.xlabel(col)\n",
    "plt.tight_layout()\n",
    "plt.show()"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "plt.figure(figsize=(10, 6))\n",
    "sns.scatterplot(data=planets_df, x='orbital_distance_au', y='density_relative', alpha=0.6)\n",
    "plt.title('Planet Density vs. Orbital Distance')\n",
    "plt.xlabel('Orbital Distance (AU)')\n",
    "plt.ylabel('Density (Relative)')\n",
    "# plt.yscale('log') # Optional log scale\n",
    "plt.ylim(0, planets_df['density_relative'].quantile(0.99)) # Zoom in, ignoring outliers\n",
    "plt.show()"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 7. Initial Model Testing"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "### 7.1 Regression: Predicting Atmospheric Density"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "Let's try to predict `atmospheric_density` based on other features like `orbital_distance_au`, `mass_earth`, `radius_earth`, and `star_mass_solar`."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Prepare data for modeling\n",
    "model_df = planets_df[['orbital_distance_au', 'mass_earth', 'radius_earth', 'star_mass_solar', 'atmospheric_density']].copy()\n",
    "model_df.dropna(inplace=True) # Drop rows with NaNs if any\n",
    "\n",
    "X = model_df[['orbital_distance_au', 'mass_earth', 'radius_earth', 'star_mass_solar']]\n",
    "y = model_df['atmospheric_density']\n",
    "\n",
    "# Split data\n",
    "X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)\n",
    "\n",
    "# Scale features (important for Linear Regression, good practice for others)\n",
    "scaler = StandardScaler()\n",
    "X_train_scaled = scaler.fit_transform(X_train)\n",
    "X_test_scaled = scaler.transform(X_test)\n",
    "\n",
    "print(f\"Training data shape: {X_train_scaled.shape}\")\n",
    "print(f\"Test data shape: {X_test_scaled.shape}\")"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Model 1: Linear Regression\n",
    "lr_model = LinearRegression()\n",
    "lr_model.fit(X_train_scaled, y_train)\n",
    "y_pred_lr = lr_model.predict(X_test_scaled)\n",
    "\n",
    "lr_mse = mean_squared_error(y_test, y_pred_lr)\n",
    "lr_r2 = r2_score(y_test, y_pred_lr)\n",
    "\n",
    "print(\"--- Linear Regression Results ---\")\n",
    "print(f\"Mean Squared Error: {lr_mse:.4f}\")\n",
    "print(f\"R-squared: {lr_r2:.4f}\")"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Model 2: Random Forest Regressor\n",
    "rf_model = RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-1)\n",
    "rf_model.fit(X_train_scaled, y_train) # Scaling doesn't hurt RF\n",
    "y_pred_rf = rf_model.predict(X_test_scaled)\n",
    "\n",
    "rf_mse = mean_squared_error(y_test, y_pred_rf)\n",
    "rf_r2 = r2_score(y_test, y_pred_rf)\n",
    "\n",
    "print(\"--- Random Forest Regressor Results ---\")\n",
    "print(f\"Mean Squared Error: {rf_mse:.4f}\")\n",
    "print(f\"R-squared: {rf_r2:.4f}\")\n",
    "\n",
    "# Feature Importances (from Random Forest)\n",
    "importances = rf_model.feature_importances_\n",
    "feature_names = X.columns\n",
    "importance_df = pd.DataFrame({'Feature': feature_names, 'Importance': importances}).sort_values(by='Importance', ascending=False)\n",
    "\n",
    "plt.figure(figsize=(8, 5))\n",
    "sns.barplot(x='Importance', y='Feature', data=importance_df)\n",
    "plt.title('Feature Importances for Predicting Atmospheric Density (Random Forest)')\n",
    "plt.show()"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Visualize Predictions vs Actuals\n",
    "plt.figure(figsize=(12, 5))\n",
    "\n",
    "plt.subplot(1, 2, 1)\n",
    "plt.scatter(y_test, y_pred_lr, alpha=0.5)\n",
    "plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], '--r', linewidth=2)\n",
    "plt.title(f'Linear Regression (R2: {lr_r2:.3f})')\n",
    "plt.xlabel('Actual Atmospheric Density')\n",
    "plt.ylabel('Predicted Atmospheric Density')\n",
    "plt.grid(True)\n",
    "\n",
    "plt.subplot(1, 2, 2)\n",
    "plt.scatter(y_test, y_pred_rf, alpha=0.5)\n",
    "plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], '--r', linewidth=2)\n",
    "plt.title(f'Random Forest (R2: {rf_r2:.3f})')\n",
    "plt.xlabel('Actual Atmospheric Density')\n",
    "plt.ylabel('Predicted Atmospheric Density')\n",
    "plt.grid(True)\n",
    "\n",
    "plt.tight_layout()\n",
    "plt.show()"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "The Random Forest model performs significantly better on this synthetic dataset, capturing more complex relationships than the linear model. Feature importance suggests mass, radius, and orbital distance are key predictors, aligning with our generation logic."
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "### 7.2 Clustering: Gaussian Mixture Model (GMM)"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "Let's use GMM to see if we can identify distinct clusters (types) of planets based on their properties (`mass_earth`, `radius_earth`, `orbital_distance_au`). This could potentially be used to generate planets belonging to specific discovered 'types'."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Select features for clustering\n",
    "cluster_features = ['mass_earth', 'radius_earth', 'orbital_distance_au']\n",
    "cluster_df = planets_df[cluster_features].copy()\n",
    "\n",
    "# Log transform skewed features for better clustering performance\n",
    "cluster_df['log_mass'] = np.log1p(cluster_df['mass_earth'])\n",
    "cluster_df['log_radius'] = np.log1p(cluster_df['radius_earth'])\n",
    "cluster_df['log_orbital_distance'] = np.log1p(cluster_df['orbital_distance_au'])\n",
    "\n",
    "X_cluster = cluster_df[['log_mass', 'log_radius', 'log_orbital_distance']]\n",
    "\n",
    "# Scale data\n",
    "scaler_cluster = StandardScaler()\n",
    "X_cluster_scaled = scaler_cluster.fit_transform(X_cluster)\n",
    "\n",
    "# Determine optimal number of clusters (e.g., using BIC)\n",
    "n_components_range = range(2, 8)\n",
    "bic_scores = []\n",
    "for n_components in n_components_range:\n",
    "    gmm = GaussianMixture(n_components=n_components, random_state=42, covariance_type='full')\n",
    "    gmm.fit(X_cluster_scaled)\n",
    "    bic_scores.append(gmm.bic(X_cluster_scaled))\n",
    "\n",
    "plt.figure(figsize=(8, 5))\n",
    "plt.plot(n_components_range, bic_scores, marker='o')\n",
    "plt.title('GMM BIC Score vs. Number of Components')\n",
    "plt.xlabel('Number of Components')\n",
    "plt.ylabel('BIC Score (Lower is better)')\n",
    "plt.grid(True)\n",
    "plt.show()\n",
    "\n",
    "optimal_n_components = n_components_range[np.argmin(bic_scores)]\n",
    "print(f\"Optimal number of components based on BIC: {optimal_n_components}\")"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Fit GMM with the optimal number of components\n",
    "gmm_final = GaussianMixture(n_components=optimal_n_components, random_state=42, covariance_type='full')\n",
    "gmm_final.fit(X_cluster_scaled)\n",
    "\n",
    "# Predict cluster labels\n",
    "cluster_labels = gmm_final.predict(X_cluster_scaled)\n",
    "planets_df['gmm_cluster'] = cluster_labels\n",
    "\n",
    "print(\"\\nCluster distribution:\")\n",
    "print(planets_df['gmm_cluster'].value_counts())"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Visualize clusters (using original scales for interpretability)\n",